# Subindo dados da camada Bronze 

## Importando dados e gerando conexão com o postgres

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import os
import numpy as np

# 2. Configurar a conexão com o banco
usuario = "meu_usuario"
senha = "minha_senha_segura"
host = "localhost"
porta = "5432"
banco = "meu_banco"

db_url = f"postgresql://{usuario}:{senha}@{host}:{porta}/{banco}"
engine = create_engine(db_url)

### Lendo dataframe olist_products com a query

In [18]:
nome_tabela = 'olist_products'

# Crie a query SQL qualificando a tabela com o schema
query = f"SELECT * FROM bronze.{nome_tabela};"

# Execute a query
# O uso do 'text()' é uma boa prática para evitar problemas de formatação com SQLAlchemy
df_query = pd.read_sql_query(text(query), engine)

# 3. Visualizar os primeiros registros do DataFrame
print("Tabela lida com sucesso com a query!")

Tabela lida com sucesso com a query!


# Fazendo agrupamento de categoria com groupby
### - Criando uma nova coluna chamada Grupo

In [ ]:
categorias_array = df_query['product_category_name'].unique()
categorias_series = pd.Series(categorias_array, name="categoria_original")

grupos = categorias_series.str.split('_').str[0]

df_categorias = pd.DataFrame({
    'categoria_original': categorias_series,
    'grupo': grupos
})

categorias_agrupadas = df_categorias.groupby('grupo')['categoria_original'].apply(list)

In [27]:
categorias_array = df_query['product_category_name'].unique()
categorias_series = pd.Series(categorias_array, name="categoria_original")
grupos = categorias_series.str.split('_').str[0]
df_categorias = pd.DataFrame({
    'categoria_original': categorias_series,
    'grupo': grupos
})

categorias_agrupadas = df_categorias.groupby('grupo')['categoria_original'].apply(list)


In [ ]:
# Assumindo que 'df_query' já existe e tem a coluna 'product_category_name'
# E que o código que você já escreveu foi executado:
categorias_array = df_query['product_category_name'].unique()
categorias_series = pd.Series(categorias_array, name="categoria_original")
grupos = categorias_series.str.split('_').str[0]
df_categorias = pd.DataFrame({
    'categoria_original': categorias_series,
    'grupo': grupos
})

# --- PASSO PRINCIPAL: JUNTAR OS DATAFRAMES ---
# Aqui nós unimos o df_query com o df_categorias
df_final = pd.merge(
    df_query,                 # Seu DataFrame original à esquerda
    df_categorias,            # Sua tabela de "mapeamento" à direita
    left_on='product_category_name', # Chave de junção no DataFrame da esquerda
    right_on='categoria_original',   # Chave de junção no DataFrame da direita
    how='left'                # Mantém todos os produtos do df_query original
)

# Exibir as primeiras linhas do novo DataFrame com a coluna "grupo"
# df_final.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,categoria_original,grupo
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumaria,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,artes,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,esporte_lazer,esporte
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,bebes,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,utilidades_domesticas,utilidades


## deletando coluna product_category_name

In [29]:
df_final = df_final.drop('product_category_name', axis=1)

### Subindo para camada Silver

In [31]:
df_final.to_sql(
        "olist_products",
        engine,
        schema="silver",  # <-- AQUI ESTÁ A MUDANÇA PRINCIPAL
        if_exists='replace',
        index=False
    )

print(f"Tabela olist_products criada com sucesso no schema silver!")

Tabela olist_products criada com sucesso no schema silver!
